In [1]:
##############################################
###   IMPORT PACKAGES
%load_ext autoreload
%autoreload 2

import random
import os as os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm 

from src.load_data import load_from_folder


In [4]:
pais = 'Mexico'
estado = '12_Guerrero' #'22_Queretaro'
municipio = 'Acapulco' #'Corregidora'

data_folder = f'data/{pais}/{estado}/{municipio}'

file_save = f'{data_folder}/final/Completo_{municipio}.csv' 

survey = load_from_folder(f'data/{pais}/surveys_demos')
padron = load_from_folder(f'{data_folder}/padron/')

In [5]:
##############################################
###         EDITE SURVEY
### Delete columns
survey = survey.drop(columns=['ID', 'Started on', 'Last updated on', 'IP address', 
                              'Status', 'Collector', 'Language', 'Device', '{fbclid}', 
                              'Nombre y Apellido', 'Numero de Documento', 'Email', 
                              'Número de Teléfono', 'Número de Celular'])

### Delete response with nan
survey = survey.dropna(subset='Podría indicarnos en ¿qué municipio del Estado de México reside actualmente?')

### Rename columns
survey = survey.rename({
    # 'IP address': 'nse',
    '¿Podría indicarnos su género?': 'genero', 
    'Para empezar, ¿podría indicarnos su edad?': 'clase',
    '¿Cuál es su máximo nivel de educación alcanzado?': 'educacion'}, axis = 1)

##############################################
########    RESPUESTAS TEXTUALES

###   GENERATE THE SEGMENT - SURVEYS
def create_segment(df2):
    survey = df2.copy()
    # # Create class
    survey.loc[survey.clase == 'Más de 58 años', 'clase_aux'] = '4'
    survey.loc[survey.clase == '43 a 58 años', 'clase_aux'] = '3'
    survey.loc[survey.clase == '27 a 42 años', 'clase_aux'] = '2'
    survey.loc[survey.clase == '18 a 26 años', 'clase_aux'] = '1'
        
    survey.loc[survey.genero == 'Femenino', 'genero_aux'] = 'F'
    survey.loc[survey.genero == 'Masculino', 'genero_aux'] = 'M'
    
    # survey.loc[survey.nse == 'AB', 'nse_aux'] = '-1'
    # survey.loc[survey.nse == 'C+', 'nse_aux'] = '-2'
    # survey.loc[survey.nse == 'C', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'C-', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'D+', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'D', 'nse_aux'] = '-4'
    # survey.loc[survey.nse == 'E', 'nse_aux'] = '-4'
    
    
    return  survey['genero_aux'] + survey['clase_aux'] #+ survey['nse_aux']

survey['segmento'] = create_segment(survey)

##############################################
###         ASSIGN RESPONSES BY SEGMENT      
survey = survey.drop(columns=[#'nse', 
    'clase', 'genero', 'educacion',
    'Podría indicarnos en ¿qué municipio del Estado de México reside actualmente?',
    #    'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'
       ])

question_list = survey.columns.values
question_list = question_list[:-1]  

dictionary = dict()
for question in question_list:
    dictionary[question] = survey.groupby('segmento')[question].apply(lambda x: x.value_counts()).unstack()
    dictionary[question] = dictionary[question].fillna(0).div(dictionary[question].fillna(0).sum(axis=1),axis=0)


#################################################
######    ASSIGN RESPONSES BY SEGMENT      ######
#################################################
padron = padron.rename({'segment' : 'segmento'}, axis=1)
segment_list = list(padron.segmento.unique())
dictionary = dict()
for question in question_list:
    dictionary[question] = survey.groupby('segmento')[question].apply(lambda x: x.value_counts()).unstack()
    dictionary[question] = dictionary[question].fillna(0).div(dictionary[question].fillna(0).sum(axis=1), axis=0)
for i in question_list:
    padron[question_list] = np.nan
for j in range(len(question_list)):
    for i in range(len(segment_list)):
        cantidad = sum(padron.segmento == segment_list[i])
        padron.loc[padron.segmento == segment_list[i], 
                   question_list[j]] = np.random.choice(dictionary[question_list[j]].columns.values.tolist(), cantidad, p = dictionary[question_list[j]].values[i].tolist())

###############################################
##### CORRECCIÓN

padron.loc[
    (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia (MORENA + PT + PVEM)') &
    (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México'),
    'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] = 'Frente Amplio por México (PRI + PAN + PRD)'

padron.loc[
    (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México (PRI + PAN + PRD)') &
    (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia'),
    'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] = 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México'

###############################################
########        Equivalencias     #############
###############################################

### Presidente 2018
padron['partido'] = padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?']
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Andrés Manuel López Obrador, de Morena', 'Voto elecciones 2018'] = 'Juntos Hacemos Historia'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'José Antonio Meade Kuribreña, del PRI', 'Voto elecciones 2018'] = 'Frente Amplio por México'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Ricardo Anaya Cortés, del PAN', 'Voto elecciones 2018'] = 'Frente Amplio por México'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Margarita Zavala, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Jaime Rodríguez Calderón, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No voto', 'Voto elecciones 2018'] = 'No votó'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto elecciones 2018'] = 'Voto en blanco'
padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No sabe', 'Voto elecciones 2018'] = 'Indecisos'

### Gobernador 2023
padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Delfina Gómez Álvarez, de Juntos Hacemos Historia por el Estado de México', 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Alejandra del Moral, de Va por el Estado de México', 'Voto gobernador 2023'] = 'Frente Amplio por México'
padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'No sabe', 'Voto gobernador 2023'] = 'No votó'
padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto gobernador 2023'] = 'Voto en blanco'

### Alcalde 2021   
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Voto alcalde 2021'] = 'MORENA'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Revolucionario Institucional (PRI)', 'Voto alcalde 2021'] = 'PRI'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Acción Nacional (PAN)', 'Voto alcalde 2021'] = 'PAN'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido de la Revolución Democrática (PRD)', 'Voto alcalde 2021'] = 'PRD'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Ciudadano (MC)', 'Voto alcalde 2021'] = 'Movimiento Ciudadano'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido del Trabajo (PT)', 'Voto alcalde 2021'] = 'PT'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Verde Ecologista de México (PVEM)', 'Voto alcalde 2021'] = 'PVEM'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Otro partido', 'Voto alcalde 2021'] = 'Indecisos'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Voto en blanco', 'Voto alcalde 2021'] = 'Voto en blanco'
padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'No voto', 'Voto alcalde 2021'] = 'Indecisos'


### Coalicion - PRESIDENTE 2024
# padron['Coalicion'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?']
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia (MORENA + PT + PVEM)', 'Coalicion'] = 'Juntos Hacemos Historia'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México (PRI + PAN + PRD)', 'Coalicion'] = 'Frente Amplio por México'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Movimiento Ciudadano (MC)', 'Coalicion'] = 'Movimiento Ciudadano'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No sabe', 'Coalicion'] = 'Indecisos'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Ninguno', 'Coalicion'] = 'Indecisos'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No va a votar', 'Coalicion'] = 'Indecisos'
    
### Candidato - PRESIDENTE 2024
# padron['Probabilidad de voto'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?']
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México', 'Probabilidad de voto'] = 'Bertha Xóchitl Gálvez Ruiz'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Marcelo Ebrard Casaubón', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Samuel Garcia, de Movimiento Ciudadano', 'Probabilidad de voto'] = 'Samuel Garcia'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Eduardo Verastegui, candidato independiente', 'Probabilidad de voto'] = 'Eduardo Verastegui'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No sabe', 'Probabilidad de voto'] = 'Indecisos'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Voto en blanco', 'Probabilidad de voto'] = 'Indecisos'
padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No va a votar', 'Probabilidad de voto'] = 'Indecisos'


### Partido - Alcalde 2024
# padron['Probabilidad de voto - Alcalde'] = padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?']
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Probabilidad de voto - Alcalde'] = 'MORENA'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Revolucionario Institucional (PRI)', 'Probabilidad de voto - Alcalde'] = 'PRI'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Acción Nacional (PAN)', 'Probabilidad de voto - Alcalde'] = 'PAN'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido de la Revolución Democrática (PRD)', 'Probabilidad de voto - Alcalde'] = 'PRD'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Ciudadano (MC)', 'Probabilidad de voto - Alcalde'] = 'Movimiento Ciudadano'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido del Trabajo (PT)', 'Probabilidad de voto - Alcalde'] = 'PT'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Verde Ecologista de México (PVEM)', 'Probabilidad de voto - Alcalde'] = 'PVEM'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'No va a votar', 'Probabilidad de voto - Alcalde'] = 'Indecisos'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Voto en blanco', 'Probabilidad de voto - Alcalde'] = 'Voto en blanco'
padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Otro partido', 'Probabilidad de voto - Alcalde'] = 'Indecisos'

### Voto duro / voto blanco
padron['Voto Duro / Voto Blando'] = 'Voto blando'
for i in range(len(padron)):
    if padron['Coalicion'][i] == 'Juntos Hacemos Historia': 
        if padron['En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?'][i] == 'Sí, lo votaría con seguridad':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Voto elecciones 2018'][i] == 'Juntos Hacemos Historia':#or padron['Voto elecciones 2018'][i] == 'Partido del Trabajo (PT)' or padron['Voto elecciones 2018'][i] == 'Partido Verde Ecologista de México (PVEM)':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
    elif padron['Coalicion'][i] == 'Frente Amplio por México':
        if padron['¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?'][i] == 'Sí, lo votaría con seguridad':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Voto elecciones 2018'][i] == 'Frente Amplio por México':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
    elif padron['Coalicion'][i] == 'Movimiento Ciudadano': 
        if padron['¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?'][i] == 'Sí, lo votaría con seguridad':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Voto elecciones 2018'][i] == 'Movimiento Ciudadano':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
            padron['Voto Duro / Voto Blando'][i] = 'Voto duro'


### Potilizado
padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento interesado', 'Politizados'] = 'Politizado'
padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento distante', 'Politizados'] = 'Despolitizado'
padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'No sabe', 'Politizados'] = 'Despolitizado'

### Posición Gob. nacional
padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno nacional', 'Posición ante el gobierno nacional'] = 'Apoya'
padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno nacional'] = 'Neutral'
padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno nacional'] = 'Neutral'
padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno nacional', 'Posición ante el gobierno nacional'] = 'No apoya'

### Posición Gob. estatal
padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno estatal', 'Posición ante el gobierno estatal'] = 'Apoya'
padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno estatal'] = 'Neutral'
padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno estatal'] = 'Neutral'
padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno estatal', 'Posición ante el gobierno estatal'] = 'No apoya'

### Posición Gob. local
padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno local', 'Posición ante el gobierno local'] = 'Apoya'
padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno local'] = 'Neutral'
padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno local'] = 'Neutral'
padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno local', 'Posición ante el gobierno local'] = 'No apoya'

### Preocupaciones generales
padron['Preocupaciones nacionales'] = padron['¿Cuál es para usted el problema más importante a resolver en México?']

### Preocupaciones locales
padron['Preocupaciones locales'] = padron['¿Cuál de los siguientes ejes debería priorizar la gestión local?']
padron.loc[padron['Preocupaciones locales'] == 'Inversión en el sistema de salud municipal', 'Preocupaciones locales'] = 'Inversión en el sistema de salud'
padron.loc[padron['Preocupaciones locales'] == 'Limpieza y recolección de basura', 'Preocupaciones locales'] = 'Limpieza y recolección de residuos'
padron.loc[padron['Preocupaciones locales'] == 'Mejora del transporte público y paradas', 'Preocupaciones locales'] = 'Mejora del transporte público'

###############################################
##### FILTROS SOCIODEMOGRÁFICOS
### Género
padron.loc[padron['genero'] == 'F', 'genero'] = 'Femenino'
padron.loc[padron['genero'] == 'M', 'genero'] = 'Masculino'

### Rango etario
padron.loc[padron['clase'] == 1, 'clase'] = '18 a 26 años - Generación Z'
padron.loc[padron['clase'] == 2, 'clase'] = '27 a 42 años - Millennials'
padron.loc[padron['clase'] == 3, 'clase'] = '43 a 58 años - Generación X'
padron.loc[padron['clase'] == 4, 'clase'] = 'Más de 58 años - Mayores'

###############################################
##### CLIMA SOCIAL
### Countinuidad
padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Continuidad', 'Continuidad'] = 'Continuidad'
padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Cambio', 'Continuidad'] = 'Cambio'
padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'No sabe', 'Continuidad'] = 'No sabe'

### Emociones
padron['Emociones'] = padron['¿Cuál es la principal emoción que le produce la situación política en la actualidad?']
padron.loc[padron['Emociones'] == 'Aversión', 'Emociones'] = 'Rechazo'

### Atributos políticos
padron['Atributos políticos'] = padron['Según su opinión, ¿Cuál es el principal atributo que valora de un político?']

### Estabilidad
padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay estabilidad', 'Estabilidad'] = 'Estabilidad'
padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay inestabilidad', 'Estabilidad'] = 'Inestabilidad'
padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'No sabe', 'Estabilidad'] = 'No sabe'

### Orden
padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay orden', 'Orden'] = 'Orden'
padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay desorden', 'Orden'] = 'Desorden'
padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'No sabe', 'Orden'] = 'No sabe'

### Presibilidad
padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento que es previsible', 'Presibilidad'] = 'Presibilidad'
padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento incertidumbre', 'Presibilidad'] = 'Incertidumbre'
padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'No sabe', 'Presibilidad'] = 'No sabe'

### Transparencia
padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Justicia', 'Justicia'] = 'Justicia'
padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Corrupción', 'Justicia'] = 'Corrupción'
padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'No sabe', 'Justicia'] = 'No sabe'

### Inseguridad
padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay seguridad', 'Seguridad'] = 'Seguridad'
padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay inseguridad', 'Seguridad'] = 'Inseguridad'
padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'No sabe', 'Seguridad'] = 'No sabe'

##############################################
###     EDIT DATA

padron = padron.rename({'random_latitude' : 'latitud', 
                        'random_longitude' : 'longitud',
                        'id_manzana' : 'cod_manzana',
                        'municipio_electoral' : 'municipio'},
                        axis=1)

padron = padron.drop(columns=[#'id_estado', 'estado', 'municipio', 'Group',
                            'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?',
                            'En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?',
                            'En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?',
                            'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?',
                            'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?',
                            'Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?',
                            'En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?',
                            '¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?',
                            '¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?',
                            '¿Qué imagen tiene de Claudia Sheinbaum Pardo?',
                            '¿Qué imagen tiene de Bertha Xóchitl Gálvez Ruiz?',
                            '¿Qué imagen tiene de Eduardo Verastegui?',
                            '¿Qué imagen tiene de Marcelo Ebrard Casaubón?',
                            '¿Qué imagen tiene de Samuel Garcia?',
                            'En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?',
                            'En relación al gobierno estatal ¿Usted se considera oficialista u opositor?',
                            'En relación al gobierno local, ¿Usted se considera oficialista u opositor?',
                            '¿Cuál es para usted el problema más importante a resolver en México?',
                            '¿Cuál de los siguientes ejes debería priorizar la gestión local?',
                            '¿Cuál es la principal emoción que le produce la situación política en la actualidad?',
                            'Según su opinión, ¿Cuál es el principal atributo que valora de un político?',
                            '¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?',
                            '¿Usted se siente interesado por la política o distante de la política?',
                            'Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?',
                            '¿Siente que en el país prevalece el orden o el desorden?',
                            '¿Siente que en el país el futuro es previsible o que hay incertidumbre?',
                            '¿Usted cree que en el país se vive una situación de justicia o corrupción?',
                            '¿Siente que hay un clima de seguridad o siente inseguridad?',
                            ' Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'])

# Convertir todas las columnas a tipo str
padron[['cod_manzana', 'latitud', 'longitud']] = padron[['cod_manzana', 'latitud', 'longitud']].astype(str)

# Reemplazar puntos por comas en la columna
padron['latitud'] = padron['latitud'].str.replace('.', ',')
padron['longitud'] = padron['longitud'].str.replace('.', ',')

# padron[['id_seccion', 'manzana']] = padron[['id_seccion', 'manzana']].astype(int)
padron['id_seccion'] = padron['id_seccion'].astype(int)
padron['cod_manzana'] = padron['cod_manzana'].apply(lambda x: f'"{x}"')

# Reemplazar puntos por comas en la columna
padron['latitud'] = padron['latitud'].str.replace('.', ',')
padron['longitud'] = padron['longitud'].str.replace('.', ',')

n = len(padron)

# Generar una lista de n valores aleatorios de 15 dígitos distintos
numeros_distintos = random.sample(range(10**14, 10**15), n)

padron.insert(0, 'id_dispositivos', numeros_distintos)

padron['nse_seccion'] = [None] * len(padron)
padron['clase_genero'] = [None] * len(padron)
padron['Voto duro - Morena'] = [None] * len(padron)
padron['Voto duro - PRI'] = [None] * len(padron)
padron['Voto duro - PAN'] = [None] * len(padron)
padron['Voto duro - PRD'] = [None] * len(padron)
padron['Voto duro - PT'] = [None] * len(padron)
padron['Voto duro - PVE'] = [None] * len(padron)
padron['Morena/AntiMorena'] = [None] * len(padron)
padron['PAN-PRI-PRD/Anti'] = [None] * len(padron)

padron['Voto Duro / Voto Blando'] = padron['Voto Duro / Voto Blando'].replace('Voto duro', 'Voto leal')
padron['Voto Duro / Voto Blando'] = padron['Voto Duro / Voto Blando'].replace('Voto blando', 'Voto switcher')


padron = padron.rename({'Voto Duro / Voto Blando' : 'Voto Leal / Voto Switcher'}, axis=1)

padron = padron[['cod_manzana', 
                 'longitud', 
                 'latitud', 
                 'nse_seccion', 
                 'nse', 
                 'clase_genero',
                 'clase', 
                 'genero', 
                 'segmento', 
                 'partido', 
                 'Voto elecciones 2018', 
                 'Coalicion', 
                 'Probabilidad de voto',
                 'Voto duro - Morena', 
                 'Voto duro - PRI', 
                 'Voto duro - PAN',
                 'Voto duro - PRD', 
                 'Voto duro - PT', 
                 'Voto duro - PVE',                          
                 'Politizados', 
                 'Posición ante el gobierno nacional',
                 'Posición ante el gobierno estatal', 
                 'Posición ante el gobierno local',
                 'Preocupaciones nacionales', 
                 'Preocupaciones locales',
                 'Emociones', 
                 'Atributos políticos', 
                 'Estabilidad', 
                 'Orden', 
                 'Presibilidad', 
                 'Justicia', 
                 'Seguridad', 
                 'Continuidad', 
                 'Morena/AntiMorena',
                 'PAN-PRI-PRD/Anti',
                 'Voto Leal / Voto Switcher', 
                 # 'Voto Duro / Voto Blando',
                 'id_dispositivos',
                 'id_seccion', 
                 'municipio',
                 'Voto gobernador 2023',
                 'Voto alcalde 2021',
                 'Probabilidad de voto - Alcalde',
                 'id_municipio_geografico',
                 'id_municipio_electoral']]



/tmp/ipykernel_12992/3984351354.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
/tmp/ipykernel_12992/3984351354.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
/tmp/ipykernel_12992/3984351354.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
/tmp/ipykernel_12992/3984351

In [6]:
padron.to_csv(file_save, index=False)

In [ ]:
# Obtener la lista de archivos en la carpeta
data_csv = [data for data in os.listdir(file) if data.endswith(".csv")]

# Recorrer cada archivo CSV en la lista
for data in tqdm(data_csv):
    # Ruta completa del archivo CSV
    route_csv = os.path.join(file, data)
    
    # Cargar el archivo CSV en un DataFrame 
    padron = pd.read_csv(route_csv)
    #padron['partido'] = 'sin datos'
    ###############################################
    # Realizar operaciones en el DataFrame 
    # padron = padron.drop(['Unnamed: 0'], axis=1)


    #################################################
    ######    ASSIGN RESPONSES BY SEGMENT      ######
    #################################################
    padron = padron.rename({'segment' : 'segmento'}, axis=1)
    segment_list = list(padron.segmento.unique())

    dictionary = dict()
    for question in question_list:
        dictionary[question] = survey.groupby('segmento')[question].apply(lambda x: x.value_counts()).unstack()
        dictionary[question] = dictionary[question].fillna(0).div(dictionary[question].fillna(0).sum(axis=1), axis=0)
    for i in question_list:
        padron[question_list] = np.nan
    for j in range(len(question_list)):
        for i in range(len(segment_list)):
            cantidad = sum(padron.segmento == segment_list[i])
            padron.loc[padron.segmento == segment_list[i],  
                    question_list[j]] = np.random.choice(dictionary[question_list[j]].columns.values.tolist(), cantidad, p = dictionary[question_list[j]].values[i].tolist())


    padron.loc[
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia ') &
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México  '),
        'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'
    ] = 'Frente Amplio por México '
    
    padron.loc[
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México ') &
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia'),
        'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'
    ] = 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México  '
    
    # padron = padron.drop('Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?', axis=1)
    

###############################################
########        Equivalencias     #############
###############################################

    ###############################################
    ##### FILTROS PREDICTIVOS

    ### Presidente 2018
    padron['partido'] = padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?']
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Andrés Manuel López Obrador, de MORENA', 'Voto elecciones 2018'] = 'Juntos Hacemos Historia'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'José Antonio Meade Kuribreña, del PRI', 'Voto elecciones 2018'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Ricardo Anaya Cortés, del PAN', 'Voto elecciones 2018'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Margarita Zavala, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Jaime Rodríguez Calderón, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No votó', 'Voto elecciones 2018'] = 'No votó'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto elecciones 2018'] = 'Voto en blanco'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No sabe', 'Voto elecciones 2018'] = 'Indecisos'

    ### Gobernador 2023
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Delfina Gómez Álvarez, de Juntos Hacemos Historia por el Estado de México', 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Alejandra del Moral, de Va por el Estado de México', 'Voto gobernador 2023'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'No votó', 'Voto gobernador 2023'] = 'No votó'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto gobernador 2023'] = 'Voto en blanco'

    ### Alcalde 2021   
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Voto alcalde 2021'] = 'MORENA'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Revolucionario Institucional (PRI)', 'Voto alcalde 2021'] = 'PRI'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Acción Nacional (PAN)', 'Voto alcalde 2021'] = 'PAN'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido de la Revolución Democrática (PRD)', 'Voto alcalde 2021'] = 'PRD'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Ciudadano (MC)', 'Voto alcalde 2021'] = 'Movimiento Ciudadano'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido del Trabajo (PT)', 'Voto alcalde 2021'] = 'PT'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Verde Ecologista de México (PVEM)', 'Voto alcalde 2021'] = 'PVEM'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Otro', 'Voto alcalde 2021'] = 'Indecisos'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Voto en blanco', 'Voto alcalde 2021'] = 'Voto en blanco'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'No sabe', 'Voto alcalde 2021'] = 'Indecisos'
                                                                                               
       
    ### Coalicion - PRESIDENTE 2024
    # padron['Coalicion'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?']
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia', 'Coalicion'] = 'Juntos Hacemos Historia'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México', 'Coalicion'] = 'Frente Amplio por México'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Movimiento Ciudadano', 'Coalicion'] = 'Movimiento Ciudadano'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No sabe', 'Coalicion'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Ninguno', 'Coalicion'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No va a votar', 'Coalicion'] = 'Indecisos'
 
    ### Candidato - PRESIDENTE 2024
    # padron['Probabilidad de voto'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?']
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México', 'Probabilidad de voto'] = 'Bertha Xóchitl Gálvez Ruiz'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Marcelo Ebrard Casaubón', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Samuel Garcia, de Movimiento Ciudadano', 'Probabilidad de voto'] = 'Samuel Garcia'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Eduardo Verastegui, candidato independiente', 'Probabilidad de voto'] = 'Eduardo Verastegui'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No sabe', 'Probabilidad de voto'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Voto en blanco', 'Probabilidad de voto'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No va a votar', 'Probabilidad de voto'] = 'Indecisos'


    ### Partido - Alcalde 2024
    # padron['Probabilidad de voto - Alcalde'] = padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?']
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Probabilidad de voto - Alcalde'] = 'MORENA'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Revolucionario Institucional (PRI)', 'Probabilidad de voto - Alcalde'] = 'PRI'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Acción Nacional (PAN)', 'Probabilidad de voto - Alcalde'] = 'PAN'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido de la Revolución Democrática (PRD)', 'Probabilidad de voto - Alcalde'] = 'PRD'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Ciudadano (MC)', 'Probabilidad de voto - Alcalde'] = 'Movimiento Ciudadano'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido del Trabajo (PT)', 'Probabilidad de voto - Alcalde'] = 'PT'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Verde Ecologista de México (PVEM)', 'Probabilidad de voto - Alcalde'] = 'PVEM'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'No va a votar', 'Probabilidad de voto - Alcalde'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Voto en blanco', 'Probabilidad de voto - Alcalde'] = 'Voto en blanco'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Otro partido', 'Probabilidad de voto - Alcalde'] = 'Indecisos'
                    

    ### Voto duro / voto blanco
    padron['Voto Duro / Voto Blando'] = 'Voto blando'
    for i in range(len(padron)):
        if padron['Coalicion'][i] == 'Juntos Hacemos Historia': 
            if padron['En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Juntos Hacemos Historia':#or padron['Voto elecciones 2018'][i] == 'Partido del Trabajo (PT)' or padron['Voto elecciones 2018'][i] == 'Partido Verde Ecologista de México (PVEM)':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Coalicion'][i] == 'Frente Amplio por México':
            if padron['¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Frente Amplio por México':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Coalicion'][i] == 'Movimiento Ciudadano': 
            if padron['¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Movimiento Ciudadano':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
         
    ### Potilizado
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento interesado', 'Politizados'] = 'Politizado'
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento distante', 'Politizados'] = 'Despolitizado'
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'No sabe', 'Politizados'] = 'Despolitizado'

    ### Posición Gob. nacional
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno nacional', 'Posición ante el gobierno nacional'] = 'Apoya'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno nacional'] = 'Neutral'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno nacional'] = 'Neutral'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno nacional', 'Posición ante el gobierno nacional'] = 'No apoya'

    ### Posición Gob. estatal
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno estatal', 'Posición ante el gobierno estatal'] = 'Apoya'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno estatal'] = 'Neutral'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno estatal'] = 'Neutral'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno estatal', 'Posición ante el gobierno estatal'] = 'No apoya'

    ### Posición Gob. local
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno local', 'Posición ante el gobierno local'] = 'Apoya'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno local'] = 'Neutral'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno local'] = 'Neutral'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno local', 'Posición ante el gobierno local'] = 'No apoya'

    ### Preocupaciones generales
    padron['Preocupaciones nacionales'] = padron['¿Cuál es para usted el problema más importante a resolver en México?']

    ### Preocupaciones locales
    padron['Preocupaciones locales'] = padron['¿Cuál de los siguientes ejes debería priorizar la gestión local?']
    padron.loc[padron['Preocupaciones locales'] == 'Inversión en el sistema de salud municipal', 'Preocupaciones locales'] = 'Inversión en el sistema de salud'
    padron.loc[padron['Preocupaciones locales'] == 'Limpieza y recolección de basura', 'Preocupaciones locales'] = 'Limpieza y recolección de residuos'
    padron.loc[padron['Preocupaciones locales'] == 'Mejora del transporte público y paradas', 'Preocupaciones locales'] = 'Mejora del transporte público'

###############################################
##### FILTROS SOCIODEMOGRÁFICOS
    ### Género
    padron.loc[padron['genero'] == 'F', 'genero'] = 'Femenino'
    padron.loc[padron['genero'] == 'M', 'genero'] = 'Masculino'

    ### Rango etario
    padron.loc[padron['clase'] == 1, 'clase'] = '18 a 26 años - Generación Z'
    padron.loc[padron['clase'] == 2, 'clase'] = '27 a 42 años - Millennials'
    padron.loc[padron['clase'] == 3, 'clase'] = '43 a 58 años - Generación X'
    padron.loc[padron['clase'] == 4, 'clase'] = 'Más de 58 años - Mayores'

###############################################
##### CLIMA SOCIAL
    ### Countinuidad
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Continuidad', 'Continuidad'] = 'Continuidad'
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Cambio', 'Continuidad'] = 'Cambio'
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'No sabe', 'Continuidad'] = 'No sabe'


    ### Emociones
    padron['Emociones'] = padron['¿Cuál es la principal emoción que le produce la situación política en la actualidad?']
    padron.loc[padron['Emociones'] == 'Aversión', 'Emociones'] = 'Rechazo'

    ### Atributos políticos
    padron['Atributos políticos'] = padron['Según su opinión, ¿Cuál es el principal atributo que valora de un político?']

    ### Estabilidad
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay estabilidad', 'Estabilidad'] = 'Estabilidad'
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay inestabilidad', 'Estabilidad'] = 'Inestabilidad'
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'No sabe', 'Estabilidad'] = 'No sabe'

    ### Orden
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay orden', 'Orden'] = 'Orden'
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay desorden', 'Orden'] = 'Desorden'
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'No sabe', 'Orden'] = 'No sabe'

    ### Presibilidad
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento que es previsible', 'Presibilidad'] = 'Presibilidad'
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento incertidumbre', 'Presibilidad'] = 'Incertidumbre'
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'No sabe', 'Presibilidad'] = 'No sabe'

    ### Transparencia
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Justicia', 'Justicia'] = 'Justicia'
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Corrupción', 'Justicia'] = 'Corrupción'
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'No sabe', 'Justicia'] = 'No sabe'

    ### Inseguridad
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay seguridad', 'Seguridad'] = 'Seguridad'
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay inseguridad', 'Seguridad'] = 'Inseguridad'
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'No sabe', 'Seguridad'] = 'No sabe'

    padron = padron.rename({'random_latitude' : 'latitud', 
                          'random_longitude' : 'longitud',
                          'id_manzana' : 'cod_manzana',
                          'municipio_electoral' : 'municipio'},
                          axis=1)

    padron = padron.drop(columns=[#'id_estado', 'estado', 'municipio', 'Group',
                                'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?',
                                'En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?',
                                'En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?',
                                'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?',
                                'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?',
                                'Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?',
                                'En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?',
                                '¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?',
                                '¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?',
                                '¿Qué imagen tiene de Claudia Sheinbaum Pardo?',
                                '¿Qué imagen tiene de Bertha Xóchitl Gálvez Ruiz?',
                                '¿Qué imagen tiene de Eduardo Verastegui?',
                                '¿Qué imagen tiene de Marcelo Ebrard Casaubón?',
                                '¿Qué imagen tiene de Samuel Garcia?',
                                'En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?',
                                'En relación al gobierno estatal ¿Usted se considera oficialista u opositor?',
                                'En relación al gobierno local, ¿Usted se considera oficialista u opositor?',
                                '¿Cuál es para usted el problema más importante a resolver en México?',
                                '¿Cuál de los siguientes ejes debería priorizar la gestión local?',
                                '¿Cuál es la principal emoción que le produce la situación política en la actualidad?',
                                'Según su opinión, ¿Cuál es el principal atributo que valora de un político?',
                                '¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?',
                                '¿Usted se siente interesado por la política o distante de la política?',
                                'Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?',
                                '¿Siente que en el país prevalece el orden o el desorden?',
                                '¿Siente que en el país el futuro es previsible o que hay incertidumbre?',
                                '¿Usted cree que en el país se vive una situación de justicia o corrupción?',
                                '¿Siente que hay un clima de seguridad o siente inseguridad?',
                                ' Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'])

    # Convertir todas las columnas a tipo str
    padron[['cod_manzana', 'latitud', 'longitud']] = padron[['cod_manzana', 'latitud', 'longitud']].astype(str)
    
    # Reemplazar puntos por comas en la columna
    padron['latitud'] = padron['latitud'].str.replace('.', ',')
    padron['longitud'] = padron['longitud'].str.replace('.', ',')
    
    # padron[['id_seccion', 'manzana']] = padron[['id_seccion', 'manzana']].astype(int)
    padron['id_seccion'] = padron['id_seccion'].astype(int)

    padron['cod_manzana'] = padron['cod_manzana'].apply(lambda x: f'"{x}"')
    
    # Reemplazar puntos por comas en la columna
    padron['latitud'] = padron['latitud'].str.replace('.', ',')
    padron['longitud'] = padron['longitud'].str.replace('.', ',')
    

    n = len(padron)

    # Generar una lista de n valores aleatorios de 15 dígitos distintos
    numeros_distintos = random.sample(range(10**14, 10**15), n)

    padron.insert(0, 'id_dispositivos', numeros_distintos)

    padron['nse_seccion'] = [None] * len(padron)
    padron['clase_genero'] = [None] * len(padron)
    padron['Voto duro - Morena'] = [None] * len(padron)
    padron['Voto duro - PRI'] = [None] * len(padron)
    padron['Voto duro - PAN'] = [None] * len(padron)
    padron['Voto duro - PRD'] = [None] * len(padron)
    padron['Voto duro - PT'] = [None] * len(padron)
    padron['Voto duro - PVE'] = [None] * len(padron)
    padron['Morena/AntiMorena'] = [None] * len(padron)
    padron['PAN-PRI-PRD/Anti'] = [None] * len(padron)
     
    padron = padron[['cod_manzana', 
                    'longitud', 
                    'latitud', 
                    'nse_seccion', 
                    'nse', 
                    'clase_genero',
                    'clase', 
                    'genero', 
                    'segmento', 
                    'partido', 
                    'Voto elecciones 2018', 
                    'Coalicion', 
                    'Probabilidad de voto',
                    'Voto duro - Morena', 
                    'Voto duro - PRI', 
                    'Voto duro - PAN',
                    'Voto duro - PRD', 
                    'Voto duro - PT', 
                    'Voto duro - PVE',                          
                    'Politizados', 
                    'Posición ante el gobierno nacional',
                    'Posición ante el gobierno estatal', 
                    'Posición ante el gobierno local',
                    'Preocupaciones nacionales', 
                    'Preocupaciones locales',
                    'Emociones', 
                    'Atributos políticos', 
                    'Estabilidad', 
                    'Orden', 
                    'Presibilidad', 
                    'Justicia', 
                    'Seguridad', 
                    'Continuidad', 
                    'Morena/AntiMorena',
                    'PAN-PRI-PRD/Anti',
                    'Voto Leal / Voto Switcher', 
                    # 'Voto Duro / Voto Blando',
                    'id_dispositivos',
                    'id_seccion', 
                    'municipio',
                    'Voto gobernador 2023',
                    'Voto alcalde 2021',
                    'Probabilidad de voto - Alcalde',
                    'id_municipio_geografico',
                    'id_municipio_electoral']]

    # padron.loc[(padron['partido'].isna()), 'partido'] = 'Voto en blanco'
    # padron.loc[(padron['Voto elecciones 2018'].isna()), 'Voto elecciones 2018'] = 'Voto en blanco'
    # padron.loc[(padron['Coalicion'].isna()), 'Coalicion'] = 'Indecisos'
    # padron.loc[(padron['Probabilidad de voto'].isna()), 'Probabilidad de voto'] = 'Indecisos'
    # padron.loc[(padron['Politizados'].isna()), 'Politizados'] = 'Politizado'
    # padron.loc[(padron['Posición ante el gobierno nacional'].isna()), 'Posición ante el gobierno nacional'] = 'Apoya'
    # padron.loc[(padron['Posición ante el gobierno estatal'].isna()), 'Posición ante el gobierno estatal'] = 'Neutral'
    # padron.loc[(padron['Posición ante el gobierno local'].isna()), 'Posición ante el gobierno local'] = 'Neutral'
    # padron.loc[(padron['Preocupaciones nacionales'].isna()), 'Preocupaciones nacionales'] = 'Inseguridad'
    # padron.loc[(padron['Preocupaciones locales'].isna()), 'Preocupaciones locales'] = 'Reparación y pavimentación de calles'
    # padron.loc[(padron['Emociones'].isna()), 'Emociones'] = 'Confianza'
    # padron.loc[(padron['Atributos políticos'].isna()), 'Atributos políticos'] = 'Honestidad'
    # padron.loc[(padron['Estabilidad'].isna()), 'Estabilidad'] = 'Estabilidad'
    # padron.loc[(padron['Orden'].isna()), 'Orden'] = 'No sabe'
    # padron.loc[(padron['Presibilidad'].isna()), 'Presibilidad'] = 'No sabe'
    # padron.loc[(padron['Justicia'].isna()), 'Justicia'] = 'Corrupción'
    # padron.loc[(padron['Seguridad'].isna()), 'Seguridad'] = 'Inseguridad'
    # padron.loc[(padron['Continuidad'].isna()), 'Continuidad'] = 'Cambio'
    # padron.loc[(padron['Voto Duro / Voto Blando'].isna()), 'Voto Duro / Voto Blando'] = 'Voto blando'
    # padron.loc[(padron['Voto gobernador 2023'].isna()), 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
    # padron.loc[(padron['Voto alcalde 2021'].isna()), 'Voto alcalde 2021'] = 'MORENA'
    # padron.loc[(padron['Probabilidad de voto - Alcalde'].isna()), 'Probabilidad de voto - Alcalde'] = 'MORENA'


    # Guardar el DataFrame modificado en un nuevo archivo CSV
    new_data = f"completo_{data}"
    route_new_csv = os.path.join(file_save, new_data)
    padron.to_csv(route_new_csv, index=False)


In [ ]:
############################################
########    RESPUESTAS NUMÉRICAS
############################################

### Delete file    
# survey.loc[survey.clase == 1, 'clase'] = None
# survey = survey.dropna(subset=['clase'])

 
###   GENERATE THE SEGMENT - SURVEYS
def create_segment(df2):
    survey = df2.copy()
    # # Create class
    survey.loc[survey.clase == 'Más de 58 años', 'clase_aux'] = '4'
    survey.loc[survey.clase == '43 a 58 años', 'clase_aux'] = '3'
    survey.loc[survey.clase == '27 a 42 años', 'clase_aux'] = '2'
    survey.loc[survey.clase == '18 a 26 años', 'clase_aux'] = '1'
        
    survey.loc[survey.genero == 'Femenino', 'genero_aux'] = 'F'
    survey.loc[survey.genero == 'Masculino', 'genero_aux'] = 'M'
    
    # survey.loc[survey.nse == 'AB', 'nse_aux'] = '-1'
    # survey.loc[survey.nse == 'C+', 'nse_aux'] = '-2'
    # survey.loc[survey.nse == 'C', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'C-', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'D+', 'nse_aux'] = '-3'
    # survey.loc[survey.nse == 'D', 'nse_aux'] = '-4'
    # survey.loc[survey.nse == 'E', 'nse_aux'] = '-4'
    
    
    return  survey['genero_aux'] + survey['clase_aux'] #+ survey['nse_aux']

survey['segmento'] = create_segment(survey)


##############################################
###         ASSIGN RESPONSES BY SEGMENT      
survey = survey.drop(columns=[#'nse', 
    'clase', 'genero', 'educacion',
    'Podría indicarnos en ¿qué municipio del Estado de México reside actualmente?',
    #    'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'
       ])

question_list = survey.columns.values
question_list = question_list[:-1]  

dictionary = dict()
for question in question_list:
    dictionary[question] = survey.groupby('segmento')[question].apply(lambda x: x.value_counts()).unstack()
    dictionary[question] = dictionary[question].fillna(0).div(dictionary[question].fillna(0).sum(axis=1),axis=0)


form = form.drop(columns=['Para empezar, ¿podría indicarnos su edad?',
       '¿Podría indicarnos su género?',
       '¿Cuál es el máximo nivel de educación completo?',
       'Podría indicarnos en ¿qué municipio del Estado de México reside actualmente?', 
    #    'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'
       ])


form.columns = question_list

##############################################

# Obtener la lista de archivos en la carpeta
data_csv = [data for data in os.listdir(file) if data.endswith(".csv")]

# Recorrer cada archivo CSV en la lista
for data in tqdm(data_csv):
    # Ruta completa del archivo CSV
    route_csv = os.path.join(file, data)
    
    # Cargar el archivo CSV en un DataFrame 
    padron = pd.read_csv(route_csv)
    #padron['partido'] = 'sin datos'
    ###############################################
    # Realizar operaciones en el DataFrame 
    # padron = padron.drop(['Unnamed: 0'], axis=1)


    #################################################
    ######    ASSIGN RESPONSES BY SEGMENT      ######
    #################################################
    padron = padron.rename({'segment' : 'segmento'}, axis=1)
    segment_list = list(padron.segmento.unique())

    dictionary = dict()
    for question in question_list:
        dictionary[question] = survey.groupby('segmento')[question].apply(lambda x: x.value_counts()).unstack()
        dictionary[question] = dictionary[question].fillna(0).div(dictionary[question].fillna(0).sum(axis=1), axis=0)
    for i in question_list:
        padron[question_list] = np.nan
    for j in range(len(question_list)):
        for i in range(len(segment_list)):
            cantidad = sum(padron.segmento == segment_list[i])
            padron.loc[padron.segmento == segment_list[i],  
                    question_list[j]] = np.random.choice(dictionary[question_list[j]].columns.values.tolist(), cantidad, p = dictionary[question_list[j]].values[i].tolist())


    # padron = padron.replace(0.0, 10.0)
    # padron = padron.replace(1.0, 0.0)
    # padron = padron.replace(2.0, 1.0)
    # padron = padron.replace(3.0, 2.0)
    # padron = padron.replace(4.0, 3.0)
    # padron = padron.replace(5.0, 4.0)
    # padron = padron.replace(6.0, 5.0)
    # padron = padron.replace(7.0, 6.0)
    # padron = padron.replace(8.0, 7.0)
    # padron = padron.replace(9.0, 8.0)
    # padron = padron.replace(10.0, 9.0)


    for question in question_list:
        for orden in range(0,10):
            answer = form[question]
            answer = answer.dropna()
            padron.loc[padron[question] == orden, question] = form[question][orden]

    padron.loc[
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia ') &
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México  '),
        'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'
    ] = 'Frente Amplio por México '
    
    padron.loc[
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México ') &
        (padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia'),
        'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'
    ] = 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México  '
    
    # padron = padron.drop('Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?', axis=1)
    

###############################################
########        Equivalencias     #############
###############################################

    ###############################################
    ##### FILTROS PREDICTIVOS

    ### Presidente 2018
    padron['partido'] = padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?']
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Andrés Manuel López Obrador, de MORENA', 'Voto elecciones 2018'] = 'Juntos Hacemos Historia'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'José Antonio Meade Kuribreña, del PRI', 'Voto elecciones 2018'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Ricardo Anaya Cortés, del PAN', 'Voto elecciones 2018'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Margarita Zavala, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Jaime Rodríguez Calderón, candidato independiente', 'Voto elecciones 2018'] = 'Indecisos'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No votó', 'Voto elecciones 2018'] = 'No votó'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto elecciones 2018'] = 'Voto en blanco'
    padron.loc[padron['En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?'] == 'No sabe', 'Voto elecciones 2018'] = 'Indecisos'

    ### Gobernador 2023
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Delfina Gómez Álvarez, de Juntos Hacemos Historia por el Estado de México', 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Alejandra del Moral, de Va por el Estado de México', 'Voto gobernador 2023'] = 'Frente Amplio por México'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'No votó', 'Voto gobernador 2023'] = 'No votó'
    padron.loc[padron['En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?'] == 'Voto en blanco', 'Voto gobernador 2023'] = 'Voto en blanco'

    ### Alcalde 2021   
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Voto alcalde 2021'] = 'MORENA'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Revolucionario Institucional (PRI)', 'Voto alcalde 2021'] = 'PRI'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Acción Nacional (PAN)', 'Voto alcalde 2021'] = 'PAN'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido de la Revolución Democrática (PRD)', 'Voto alcalde 2021'] = 'PRD'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Movimiento Ciudadano (MC)', 'Voto alcalde 2021'] = 'Movimiento Ciudadano'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido del Trabajo (PT)', 'Voto alcalde 2021'] = 'PT'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Partido Verde Ecologista de México (PVEM)', 'Voto alcalde 2021'] = 'PVEM'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Otro', 'Voto alcalde 2021'] = 'Indecisos'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'Voto en blanco', 'Voto alcalde 2021'] = 'Voto en blanco'
    padron.loc[padron['En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?'] == 'No sabe', 'Voto alcalde 2021'] = 'Indecisos'
                                                                                               
       
    ### Coalicion - PRESIDENTE 2024
    # padron['Coalicion'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?']
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Juntos Hacemos Historia', 'Coalicion'] = 'Juntos Hacemos Historia'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Frente Amplio por México', 'Coalicion'] = 'Frente Amplio por México'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Movimiento Ciudadano', 'Coalicion'] = 'Movimiento Ciudadano'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No sabe', 'Coalicion'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'Ninguno', 'Coalicion'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'] == 'No va a votar', 'Coalicion'] = 'Indecisos'
 
    ### Candidato - PRESIDENTE 2024
    # padron['Probabilidad de voto'] = padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?']
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Claudia Sheinbaum Pardo, Juntos Hacemos Historia', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Bertha Xóchitl Gálvez Ruiz, del Frente Amplio por México', 'Probabilidad de voto'] = 'Bertha Xóchitl Gálvez Ruiz'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Marcelo Ebrard Casaubón', 'Probabilidad de voto'] = 'Claudia Sheinbaum Pardo'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Samuel Garcia, de Movimiento Ciudadano', 'Probabilidad de voto'] = 'Samuel Garcia'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Eduardo Verastegui, candidato independiente', 'Probabilidad de voto'] = 'Eduardo Verastegui'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No sabe', 'Probabilidad de voto'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'Voto en blanco', 'Probabilidad de voto'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'] == 'No va a votar', 'Probabilidad de voto'] = 'Indecisos'


    ### Partido - Alcalde 2024
    # padron['Probabilidad de voto - Alcalde'] = padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?']
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Regeneración Nacional (MORENA)', 'Probabilidad de voto - Alcalde'] = 'MORENA'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Revolucionario Institucional (PRI)', 'Probabilidad de voto - Alcalde'] = 'PRI'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Acción Nacional (PAN)', 'Probabilidad de voto - Alcalde'] = 'PAN'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido de la Revolución Democrática (PRD)', 'Probabilidad de voto - Alcalde'] = 'PRD'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Movimiento Ciudadano (MC)', 'Probabilidad de voto - Alcalde'] = 'Movimiento Ciudadano'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido del Trabajo (PT)', 'Probabilidad de voto - Alcalde'] = 'PT'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Partido Verde Ecologista de México (PVEM)', 'Probabilidad de voto - Alcalde'] = 'PVEM'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'No va a votar', 'Probabilidad de voto - Alcalde'] = 'Indecisos'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Voto en blanco', 'Probabilidad de voto - Alcalde'] = 'Voto en blanco'
    padron.loc[padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'] == 'Otro partido', 'Probabilidad de voto - Alcalde'] = 'Indecisos'
                    

    ### Voto duro / voto blanco
    padron['Voto Duro / Voto Blando'] = 'Voto blando'
    for i in range(len(padron)):
        if padron['Coalicion'][i] == 'Juntos Hacemos Historia': 
            if padron['En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Juntos Hacemos Historia':#or padron['Voto elecciones 2018'][i] == 'Partido del Trabajo (PT)' or padron['Voto elecciones 2018'][i] == 'Partido Verde Ecologista de México (PVEM)':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Coalicion'][i] == 'Frente Amplio por México':
            if padron['¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Frente Amplio por México':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
        elif padron['Coalicion'][i] == 'Movimiento Ciudadano': 
            if padron['¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?'][i] == 'Sí, lo votaría con seguridad':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
            elif padron['Voto elecciones 2018'][i] == 'Movimiento Ciudadano':# or padron['Voto elecciones 2018'][i] == 'PRD' or padron['Voto elecciones 2018'][i] == 'PAN':
                padron['Voto Duro / Voto Blando'][i] = 'Voto duro'
         
    ### Potilizado
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento interesado', 'Politizados'] = 'Politizado'
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'Me siento distante', 'Politizados'] = 'Despolitizado'
    padron.loc[padron['¿Usted se siente interesado por la política o distante de la política?'] == 'No sabe', 'Politizados'] = 'Despolitizado'

    ### Posición Gob. nacional
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno nacional', 'Posición ante el gobierno nacional'] = 'Apoya'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno nacional'] = 'Neutral'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno nacional'] = 'Neutral'
    padron.loc[padron['En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno nacional', 'Posición ante el gobierno nacional'] = 'No apoya'

    ### Posición Gob. estatal
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno estatal', 'Posición ante el gobierno estatal'] = 'Apoya'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno estatal'] = 'Neutral'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno estatal'] = 'Neutral'
    padron.loc[padron['En relación al gobierno estatal ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno estatal', 'Posición ante el gobierno estatal'] = 'No apoya'

    ### Posición Gob. local
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Apoyo al gobierno local', 'Posición ante el gobierno local'] = 'Apoya'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Ni apoyo ni soy opositor', 'Posición ante el gobierno local'] = 'Neutral'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'No sabe', 'Posición ante el gobierno local'] = 'Neutral'
    padron.loc[padron['En relación al gobierno local, ¿Usted se considera oficialista u opositor?'] == 'Estoy en la oposición al gobierno local', 'Posición ante el gobierno local'] = 'No apoya'

    ### Preocupaciones generales
    padron['Preocupaciones nacionales'] = padron['¿Cuál es para usted el problema más importante a resolver en México?']

    ### Preocupaciones locales
    padron['Preocupaciones locales'] = padron['¿Cuál de los siguientes ejes debería priorizar la gestión local?']
    padron.loc[padron['Preocupaciones locales'] == 'Inversión en el sistema de salud municipal', 'Preocupaciones locales'] = 'Inversión en el sistema de salud'
    padron.loc[padron['Preocupaciones locales'] == 'Limpieza y recolección de basura', 'Preocupaciones locales'] = 'Limpieza y recolección de residuos'
    padron.loc[padron['Preocupaciones locales'] == 'Mejora del transporte público y paradas', 'Preocupaciones locales'] = 'Mejora del transporte público'

###############################################
##### FILTROS SOCIODEMOGRÁFICOS
    ### Género
    padron.loc[padron['genero'] == 'F', 'genero'] = 'Femenino'
    padron.loc[padron['genero'] == 'M', 'genero'] = 'Masculino'

    ### Rango etario
    padron.loc[padron['clase'] == 0, 'clase'] = '18 a 26 años - Generación Z'
    padron.loc[padron['clase'] == 1, 'clase'] = '27 a 42 años - Millennials'
    padron.loc[padron['clase'] == 2, 'clase'] = '43 a 58 años - Generación X'
    padron.loc[padron['clase'] == 3, 'clase'] = 'Más de 58 años - Mayores'

###############################################
##### CLIMA SOCIAL
    ### Countinuidad
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Continuidad', 'Continuidad'] = 'Continuidad'
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'Cambio', 'Continuidad'] = 'Cambio'
    padron.loc[padron['¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?'] == 'No sabe', 'Continuidad'] = 'No sabe'


    ### Emociones
    padron['Emociones'] = padron['¿Cuál es la principal emoción que le produce la situación política en la actualidad?']
    padron.loc[padron['Emociones'] == 'Aversión', 'Emociones'] = 'Rechazo'

    ### Atributos políticos
    padron['Atributos políticos'] = padron['Según su opinión, ¿Cuál es el principal atributo que valora de un político?']

    ### Estabilidad
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay estabilidad', 'Estabilidad'] = 'Estabilidad'
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'Siento que hay inestabilidad', 'Estabilidad'] = 'Inestabilidad'
    padron.loc[padron['Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?'] == 'No sabe', 'Estabilidad'] = 'No sabe'

    ### Orden
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay orden', 'Orden'] = 'Orden'
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'Siento que hay desorden', 'Orden'] = 'Desorden'
    padron.loc[padron['¿Siente que en el país prevalece el orden o el desorden?'] == 'No sabe', 'Orden'] = 'No sabe'

    ### Presibilidad
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento que es previsible', 'Presibilidad'] = 'Presibilidad'
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'Siento incertidumbre', 'Presibilidad'] = 'Incertidumbre'
    padron.loc[padron['¿Siente que en el país el futuro es previsible o que hay incertidumbre?'] == 'No sabe', 'Presibilidad'] = 'No sabe'

    ### Transparencia
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Justicia', 'Justicia'] = 'Justicia'
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'Corrupción', 'Justicia'] = 'Corrupción'
    padron.loc[padron['¿Usted cree que en el país se vive una situación de justicia o corrupción?'] == 'No sabe', 'Justicia'] = 'No sabe'

    ### Inseguridad
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay seguridad', 'Seguridad'] = 'Seguridad'
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'Siento que hay inseguridad', 'Seguridad'] = 'Inseguridad'
    padron.loc[padron['¿Siente que hay un clima de seguridad o siente inseguridad?'] == 'No sabe', 'Seguridad'] = 'No sabe'

    padron = padron.rename({'random_latitude' : 'latitud', 
                          'random_longitude' : 'longitud',
                          'id_manzana' : 'cod_manzana'},
                          axis=1)

    padron = padron.drop(columns=[#'id_estado', 'estado', 'municipio', 'Group',
                                'En las elecciones presidenciales de 2018, ¿Recuerda por quién votó?',
                                'En las elecciones a gobernador de 2023, ¿Recuerda por quién votó?',
                                'En las elecciones a alcalde del 2021, ¿Recuerda a qué partido votó?',
                                'Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?',
                                'Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?',
                                'Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?',
                                'En relación a las distintas propuestas políticas, ¿Usted podría votar a un candidato de la coalición Juntos Hacemos Historia como presidente?',
                                '¿Usted podría votar a un candidato de la coalición Frente Amplio por México como presidente?',
                                '¿Usted podría votar a un candidato de Movimiento Ciudadano como presidente?',
                                '¿Qué imagen tiene de Claudia Sheinbaum Pardo?',
                                '¿Qué imagen tiene de Bertha Xóchitl Gálvez Ruiz?',
                                '¿Qué imagen tiene de Eduardo Verastegui?',
                                '¿Qué imagen tiene de Marcelo Ebrard Casaubón?',
                                '¿Qué imagen tiene de Samuel Garcia?',
                                'En relación al gobierno nacional, ¿Usted se considera oficialista u opositor?',
                                'En relación al gobierno estatal ¿Usted se considera oficialista u opositor?',
                                'En relación al gobierno local, ¿Usted se considera oficialista u opositor?',
                                '¿Cuál es para usted el problema más importante a resolver en México?',
                                '¿Cuál de los siguientes ejes debería priorizar la gestión local?',
                                '¿Cuál es la principal emoción que le produce la situación política en la actualidad?',
                                'Según su opinión, ¿Cuál es el principal atributo que valora de un político?',
                                '¿Usted se encuentra a favor de la continuidad o del cambio del gobierno?',
                                '¿Usted se siente interesado por la política o distante de la política?',
                                'Pensando en la situación actual del país, ¿usted diría que hay estabilidad o inestabilidad?',
                                '¿Siente que en el país prevalece el orden o el desorden?',
                                '¿Siente que en el país el futuro es previsible o que hay incertidumbre?',
                                '¿Usted cree que en el país se vive una situación de justicia o corrupción?',
                                '¿Siente que hay un clima de seguridad o siente inseguridad?',
                                ' Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'])

    # Convertir todas las columnas a tipo str
    padron[['cod_manzana', 'latitud', 'longitud']] = padron[['cod_manzana', 'latitud', 'longitud']].astype(str)
    
    # Reemplazar puntos por comas en la columna
    padron['latitud'] = padron['latitud'].str.replace('.', ',')
    padron['longitud'] = padron['longitud'].str.replace('.', ',')
    
    # padron[['id_seccion', 'manzana']] = padron[['id_seccion', 'manzana']].astype(int)
    padron['id_seccion'] = padron['id_seccion'].astype(int)

    padron['cod_manzana'] = padron['cod_manzana'].apply(lambda x: f'"{x}"')
    
    # Reemplazar puntos por comas en la columna
    padron['latitud'] = padron['latitud'].str.replace('.', ',')
    padron['longitud'] = padron['longitud'].str.replace('.', ',')
    
    import random

    n = len(padron)

    # Generar una lista de n valores aleatorios de 15 dígitos distintos
    numeros_distintos = random.sample(range(10**14, 10**15), n)

    padron.insert(0, 'id_dispositivos', numeros_distintos)

    padron['nse_seccion'] = [None] * len(padron)
    padron['clase_genero'] = [None] * len(padron)
    padron['Voto duro - Morena'] = [None] * len(padron)
    padron['Voto duro - PRI'] = [None] * len(padron)
    padron['Voto duro - PAN'] = [None] * len(padron)
    padron['Voto duro - PRD'] = [None] * len(padron)
    padron['Voto duro - PT'] = [None] * len(padron)
    padron['Voto duro - PVE'] = [None] * len(padron)
    padron['Morena/AntiMorena'] = [None] * len(padron)
    padron['PAN-PRI-PRD/Anti'] = [None] * len(padron)
    padron['municipio'] = [None] * len(padron)
     
    padron = padron[[ 'cod_manzana', 
                    'longitud', 
                    'latitud', 
                    'nse_seccion', 
                    'nse', 
                    'clase_genero',
                    'clase', 
                    'genero', 
                    'segmento', 
                    'partido', 
                    'Voto elecciones 2018', 
                    'Coalicion', 
                    'Probabilidad de voto',
                    'Voto duro - Morena', 
                    'Voto duro - PRI', 
                    'Voto duro - PAN',
                    'Voto duro - PRD', 
                    'Voto duro - PT', 
                    'Voto duro - PVE',
                    'Politizados', 
                    'Posición ante el gobierno nacional',
                    'Posición ante el gobierno estatal', 
                    'Posición ante el gobierno local',
                    'Preocupaciones nacionales', 
                    'Preocupaciones locales',
                    'Emociones', 
                    'Atributos políticos', 
                    'Estabilidad', 
                    'Orden', 
                    'Presibilidad', 
                    'Justicia', 
                    'Seguridad', 
                    'Continuidad', 
                    'Morena/AntiMorena',
                    'PAN-PRI-PRD/Anti',
                    'Voto Duro / Voto Blando', 
                    'id_dispositivos',
                    'id_seccion', 
                    'municipio',
                    'Voto gobernador 2023',
                    'Voto alcalde 2021',
                    'Probabilidad de voto - Alcalde']]

    padron.loc[(padron['partido'].isna()), 'partido'] = 'Voto en blanco'
    padron.loc[(padron['Voto elecciones 2018'].isna()), 'Voto elecciones 2018'] = 'Voto en blanco'
    padron.loc[(padron['Coalicion'].isna()), 'Coalicion'] = 'Indecisos'
    padron.loc[(padron['Probabilidad de voto'].isna()), 'Probabilidad de voto'] = 'Indecisos'
    padron.loc[(padron['Politizados'].isna()), 'Politizados'] = 'Politizado'
    padron.loc[(padron['Posición ante el gobierno nacional'].isna()), 'Posición ante el gobierno nacional'] = 'Apoya'
    padron.loc[(padron['Posición ante el gobierno estatal'].isna()), 'Posición ante el gobierno estatal'] = 'Neutral'
    padron.loc[(padron['Posición ante el gobierno local'].isna()), 'Posición ante el gobierno local'] = 'Neutral'
    padron.loc[(padron['Preocupaciones nacionales'].isna()), 'Preocupaciones nacionales'] = 'Inseguridad'
    padron.loc[(padron['Preocupaciones locales'].isna()), 'Preocupaciones locales'] = 'Reparación y pavimentación de calles'
    padron.loc[(padron['Emociones'].isna()), 'Emociones'] = 'Confianza'
    padron.loc[(padron['Atributos políticos'].isna()), 'Atributos políticos'] = 'Honestidad'
    padron.loc[(padron['Estabilidad'].isna()), 'Estabilidad'] = 'Estabilidad'
    padron.loc[(padron['Orden'].isna()), 'Orden'] = 'No sabe'
    padron.loc[(padron['Presibilidad'].isna()), 'Presibilidad'] = 'No sabe'
    padron.loc[(padron['Justicia'].isna()), 'Justicia'] = 'Corrupción'
    padron.loc[(padron['Seguridad'].isna()), 'Seguridad'] = 'Inseguridad'
    padron.loc[(padron['Continuidad'].isna()), 'Continuidad'] = 'Cambio'
    padron.loc[(padron['Voto Duro / Voto Blando'].isna()), 'Voto Duro / Voto Blando'] = 'Voto blando'
    padron.loc[(padron['Voto gobernador 2023'].isna()), 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
    padron.loc[(padron['Voto alcalde 2021'].isna()), 'Voto alcalde 2021'] = 'MORENA'
    padron.loc[(padron['Probabilidad de voto - Alcalde'].isna()), 'Probabilidad de voto - Alcalde'] = 'MORENA'


    # Guardar el DataFrame modificado en un nuevo archivo CSV
    new_data = f"completo_{data}"
    route_new_csv = os.path.join(file_save, new_data)
    padron.to_csv(route_new_csv, index=False)



    # # # # Guardar el DataFrame en formato XLSX con un nombre de hoja personalizado
    # # ruta_archivo = f"Padron Ciudad de Mexico Final {data}.xlsx"
    # # nombre_hoja = f'CDMX {data}'  # Nombre de la hoja personalizado
    # # padron.to_excel(ruta_archivo, index=False, sheet_name=nombre_hoja)



# ### Guardar el DataFrame modificado en un nuevo archivo CSV
# new_data = f"predictor_{data}"
# route_new_csv = os.path.join(file_save, new_data)
# padron.to_csv(route_new_csv, index=False)

In [ ]:
# 'Entre $22.000 y $30.000', 'Entre $4.000 a $8.000',
# 'Entre $12.000 y $17.000', 'Entre $8.000 a $12.000',
# 'Entre $17.000 y $22.000', 'Menos de $4.000', 'Más de $30.000'

# padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'].unique()
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Menos de $4.000', 'NSE'] = 'E'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Entre $4.000 a $8.000', 'NSE'] = 'D'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Entre $8.000 a $12.000', 'NSE'] = 'D+'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Entre $12.000 y $17.000', 'NSE'] = 'C-'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Entre $17.000 y $22.000', 'NSE'] = 'C'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Entre $22.000 y $30.000', 'NSE'] = 'C+'
# padron.loc[padron['Para finalizar, ¿Cuál de las siguientes opciones representa mejor el ingreso bruto anual de su hogar?'] == 'Más de $30.000', 'NSE'] = 'AB'


In [ ]:
columns = padron[[   'partido', 
                    'Voto elecciones 2018', 
                    'Coalicion', 
                    'Probabilidad de voto',
                    'Politizados', 
                    'Posición ante el gobierno nacional',
                    'Posición ante el gobierno estatal', 
                    'Posición ante el gobierno local',
                    'Preocupaciones nacionales', 
                    'Preocupaciones locales',
                    'Emociones', 
                    'Atributos políticos', 
                    'Estabilidad', 
                    'Orden', 
                    'Presibilidad', 
                    'Justicia', 
                    'Seguridad', 
                    'Continuidad', 
                    'Voto Duro / Voto Blando', 
                    'Voto gobernador 2023',
                    'Voto alcalde 2021',
                    'Probabilidad de voto - Alcalde']]

for column in columns:
    most_common = padron[column].mode()[0]
    print(f"Columna '{column}': El valor más repetido es '{most_common}'")

In [ ]:
padron.loc[(padron['partido'].isna()), 'partido'] = 'Voto en blanco'
padron.loc[(padron['Voto elecciones 2018'].isna()), 'Voto elecciones 2018'] = 'Voto en blanco'
padron.loc[(padron['Coalicion'] == None), 'Coalicion'] = 'Indecisos'
padron.loc[(padron['Probabilidad de voto'].isna()), 'Probabilidad de voto'] = 'Indecisos'
padron.loc[(padron['Politizados'].isna()), 'Politizados'] = 'Politizado'
padron.loc[(padron['Posición ante el gobierno nacional'].isna()), 'Posición ante el gobierno nacional'] = 'Apoya'
padron.loc[(padron['Posición ante el gobierno estatal'].isna()), 'Posición ante el gobierno estatal'] = 'Neutral'
padron.loc[(padron['Posición ante el gobierno local'].isna()), 'Posición ante el gobierno local'] = 'Neutral'
padron.loc[(padron['Preocupaciones nacionales'].isna()), 'Preocupaciones nacionales'] = 'Inseguridad'
padron.loc[(padron['Preocupaciones locales'].isna()), 'Preocupaciones locales'] = 'Reparación y pavimentación de calles'
padron.loc[(padron['Emociones'].isna()), 'Emociones'] = 'Ira'
padron.loc[(padron['Atributos políticos'].isna()), 'Atributos políticos'] = 'Honestidad'
padron.loc[(padron['Estabilidad'].isna()), 'Estabilidad'] = 'Estabilidad'
padron.loc[(padron['Orden'].isna()), 'Orden'] = 'No sabe'
padron.loc[(padron['Presibilidad'].isna()), 'Presibilidad'] = 'No sabe'
padron.loc[(padron['Justicia'].isna()), 'Justicia'] = 'Corrupción'
padron.loc[(padron['Seguridad'].isna()), 'Seguridad'] = 'Inseguridad'
padron.loc[(padron['Continuidad'].isna()), 'Continuidad'] = 'Cambio'
padron.loc[(padron['Voto Duro / Voto Blando'].isna()), 'Voto Duro / Voto Blando'] = 'Voto blando'
padron.loc[(padron['Voto gobernador 2023'].isna()), 'Voto gobernador 2023'] = 'Juntos Hacemos Historia'
padron.loc[(padron['Voto alcalde 2021'].isna()), 'Voto alcalde 2021'] = 'MORENA'
padron.loc[(padron['Probabilidad de voto - Alcalde'].isna()), 'Probabilidad de voto - Alcalde'] = 'MORENA'


In [ ]:
 display(padron['Si las elecciones a presidente fueran hoy, ¿A cuál de las siguientes coaliciones votaría?'].value_counts())
 display(padron['Si las elecciones a presidente fueran hoy, ¿A cuál de los siguientes candidatos votaría?'].value_counts())
 display(padron['Si las elecciones a alcalde fueran hoy, ¿A cuál de los siguientes partidos votaría?'].value_counts())

In [ ]:
# # ####################################
# # ## Caracteristicas
# # 'id_dispositivo'
# # 'longitud'
# # 'latitud',
# # 'id_manzana', 
# # 'id_municipio', ## municipio al cual pertenece la manzana
# # 'municipio', ## municipio al cual pertenece la manzana 
# # 'id_seccion',
# # 'id_municipio_seccion', ## municipio al cual pertenece la sección electoral
# # 'municipio_seccion', ## municipio al cual pertenece la sección electoral
# # 'id_estado'
# # 'estado'
# # 'id_pais'
# # 'pais'
# # 'clase', ## rango etario
# # 'genero', 
# # 'nse'
# # ####################################
# # ## Politica
# # 'resultado_nacional' ## resultado anterior elección a presidente 
# # 'resultado_provincial' ## resultado anterior elección a gobernador
# # 'resultado_local' ## resultado anterior elección a alcalde / intendente
# # ##
# # 'proyeccion_colaicion_nacional' ## proyeccion de voto a presidente por coalición
# # 'proyeccion_partido_nacional' ## proyeccion de voto a presidente por partido
# # 'proyeccion_candidato_nacional' ## proyeccion de voto a presidente por candidato
# # ##
# # 'proyeccion_colaicion_provincial' ## proyeccion de voto a gobernador por coalición
# # 'proyeccion_partido_provincial' ## proyeccion de voto a gobernador por partido
# # 'proyeccion_candidato_provincial' ## proyeccion de voto a gobernador por candidato
# # ##
# # 'proyeccion_colaicion_local' ## proyeccion de voto a alcalde/intendente por coalición
# # 'proyeccion_partido_local' ## proyeccion de voto a alcalde/intendente por partido
# # 'proyeccion_candidato_local' ## proyeccion de voto a alcalde/intendente por candidato

# # ####################################
# # ## Opinión política
# # 'votoduro', ## voto duro o voto blando 
# # 'politizados', 
# # 'posicion_nacional',  # posición ante el gobierno nacional
# # 'posicion_estatal',   # posición ante el gobierno estatal
# # 'posicion_local',  # posición ante el gobierno local
# # 'preocupaciones_nacionales', 
# # 'preocupaciones_locales',

# # ####################################
# # ## Clima social
# # 'emociones', 
# # 'atributos_politicos', 
# # 'continuidad', 
# # 'estabilidad', 
# # 'orden', 
# # 'justicia', 
# # 'seguridad', 
# # 'presibilidad', 
# # ## pueden haber más opciones en esa sección